<a href="https://colab.research.google.com/github/Jin-K-Yang/Clustering-by-Louvain-Algorithm/blob/main/louvain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 定義函示

In [12]:
import community as community_louvain
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
import copy
from glob import glob

def create_graph(filename):
  # load one excel file
  df = pd.read_excel(filename, usecols=['會員編號', '部門編號'], converters={'部門編號':str})
  df = df.dropna(how='any',axis=0) 

  # convert into two array which represent customer and product
  array = df.to_numpy()
  split_array = np.hsplit(array, 2)
  customer = split_array[0]
  product = split_array[1]

  # construct bipartite graph
  G = nx.Graph()
  for i in range(len(product)):
      # zero padding
      product[i][0] = product[i][0].zfill(5)

      # add nodes
      G.add_nodes_from(product[i], bipartite = 0)
      G.add_nodes_from(customer[i], bipartite = 1)
      
      # add edges
      if G.has_edge(product[i][0], customer[i][0]):
          G[product[i][0]][customer[i][0]]['weight'] += 1
      else:
          G.add_edge(product[i][0], customer[i][0], weight = 1)
  return G

# folder version
def create_graph_folder(filename):
  # load multiple excel file
  files = glob(filename)
  df = pd.concat([pd.read_excel(f, usecols=['會員編號', '部門編號'], converters={'部門編號':str}) for f in files])
  df = df.dropna(how='any',axis=0) 

  # convert into two array which represent customer and product
  array = df.to_numpy()
  split_array = np.hsplit(array, 2)
  customer = split_array[0]
  product = split_array[1]

  # construct bipartite graph
  G = nx.Graph()
  for i in range(len(product)):
      # zero padding
      product[i][0] = product[i][0].zfill(5)

      # add nodes
      G.add_nodes_from(product[i], bipartite = 0)
      G.add_nodes_from(customer[i], bipartite = 1)
      
      # add edges
      if G.has_edge(product[i][0], customer[i][0]):
          G[product[i][0]][customer[i][0]]['weight'] += 1
      else:
          G.add_edge(product[i][0], customer[i][0], weight = 1)
  return G

def louvain(G):
  # show the total number of nodes and edges
  # print(G.number_of_nodes())
  # print(G.number_of_edges())

  # compute the best partition
  partition = community_louvain.best_partition(G)
  # print(partition)

  # classify the result
  community = {}
  for key, value in set(partition.items()):
      if key[0].isalpha():
          community.setdefault(value, {}).setdefault('customer', []).append(key)
      else:
          community.setdefault(value, {}).setdefault('product', []).append(key)

  # calculate the sum of edges of clusters
  for i in range(len(community)):
    SG = G.subgraph(community[i]['product'] + community[i]['customer'])
    community[i]['edges_sum'] = SG.number_of_edges()

  return community
  """
  total_name=''
  for i in range(len(community)):
    print(i)
    for j in community[i]['product']:
      total_name += num_to_product(j)
      total_name += ' , '
    print(total_name)
    total_name=''
  """

# benefit-cost analysis
def benefit_cost(community):
  benefit = {
      'Customer_Count':[],
      'Product_Count':[],
      'Cost':[],
      'Benefit':[],
      'Ratio':[],
  }

  total = {
      'Customer_Count':0,
      'Product_Count':0,
      'Cost':0,
      'Benefit':0,
      'Ratio':0,
  }

  for i in range(len(community)):
      benefit['Customer_Count'].append(len(community[i]['customer']))
      benefit['Product_Count'].append(len(community[i]['product']))
      benefit['Cost'].append(len(community[i]['customer']) * len(community[i]['product']))
      benefit['Benefit'].append(community[i]['edges_sum'])
      benefit['Ratio'].append(community[i]['edges_sum'] / (len(community[i]['customer']) * len(community[i]['product'])))

      total['Customer_Count'] += len(community[i]['customer'])
      total['Product_Count'] += len(community[i]['product'])
      total['Cost'] += len(community[i]['customer']) * len(community[i]['product'])
      total['Benefit'] += community[i]['edges_sum']

  total['Ratio'] = total['Benefit'] / total['Cost']

  df_total = pd.DataFrame(total, index=['total'])
  df_benefit = pd.DataFrame(benefit)

  result = pd.concat([df_benefit, df_total])
  return result

# map the department number and deparment name from excel file
def map_department(file):
  map_df = pd.read_excel(file, usecols=['部門名稱', '部門編號'], converters={'部門編號':str})
  map_array = np.hsplit(map_df.to_numpy(), 2)
  map = {}

  for i in range(len(map_array[0])):
    map[map_array[0][i][0].zfill(5)] = map_array[1][i][0]
  return map

# show the product name in each clusters
def show_product(map, community):
  temp = copy.deepcopy(community)
  for i in range(len(temp)):
    for j in range(len(temp[i]['product'])):
      temp[i]['product'][j] = map[temp[i]['product'][j]]
    print(temp[i]['product'])
  return temp

# 建立Excel

In [13]:
# write result to the excel file
import openpyxl
from openpyxl import load_workbook

# You can change the file address below this line
total_result_file_name = '/content/drive/Shareddrives//louvain/test_result.xlsx'

def write_clusters(file_name, sheet_name, result):
  wb = load_workbook(filename = file_name)
  sheet = wb[sheet_name]
  for i in range(len(result)):
    for j in range(len(result[i]['product'])):
      sheet.cell(row=i+2, column=7+j).value = result[i]['product'][j]
    # write the customer data into the excel file
    for k in range(len(result[i]['customer'])):
      sheet.cell(row=30, column=i+1).value = '分群' + str(i)
      sheet.cell(row=k+31, column=i+1).value = result[i]['customer'][k]
  wb.save(file_name)
  print('All clusters have benn written in file.')

def write_transaction(file_name, sheet_name, G, map):
  wb = load_workbook(filename = file_name)
  sheet = wb.create_sheet(sheet_name)
  sheet.cell(row=1, column=1).value = '顧客'
  sheet.cell(row=1, column=2).value = '產品'
  sheet.cell(row=1, column=3).value = '購買次數'

  for i, e in enumerate(G.edges.data("weight", default=1)):
    if e[0][0].isalpha():
      sheet.cell(row=2+i, column=1).value = e[0]
      sheet.cell(row=2+i, column=2).value = map[e[1]]
    else:
      sheet.cell(row=2+i, column=1).value = e[1]
      sheet.cell(row=2+i, column=2).value = map[e[0]]
    sheet.cell(row=2+i, column=3).value = e[2]
  wb.save(file_name)
  print('The transactions of {index} have been written in file.'.format(index=sheet_name))

# with pd.ExcelWriter(total_result_file_name) as writer:  
#   benefit_cost_2018_2021.to_excel(writer, sheet_name='3年(20180601-20210531)')
#   benefit_cost_2016_2021.to_excel(writer, sheet_name='5年(20160601-20210531)')
#   benefit_cost_2014_2021.to_excel(writer, sheet_name='7年(20140601-20210531)')

# write_clusters(total_result_file_name, '3年(20180601-20210531)', result_2018_2021)
# write_clusters(total_result_file_name, '5年(20160601-20210531)', result_2016_2021)
# write_clusters(total_result_file_name, '7年(20140601-20210531)', result_2014_2021)

# 分析

In [16]:
# option argument for folder, if it is True then the function will proccess all file in input address.
def analysis(input_file, output_file, folder = False):
  sheet_name = 'benefit_cost'

  if folder == False:
    G = create_graph(input_file)
  else:
    G = create_graph_folder(input_file)

  result = louvain(G)
  benefit = benefit_cost(result)
  with pd.ExcelWriter(output_file) as writer:  
    benefit.to_excel(writer, sheet_name=sheet_name)

  map = map_department('/content/drive/Shareddrives//louvain/department_list.xlsx')
  map_result = show_product(map, result)

  write_clusters(output_file, sheet_name, map_result)
  for i in range(len(result)):
    SG = G.subgraph(result[i]['product'] + result[i]['customer'])
    write_transaction(output_file, 'Cluster {index}'.format(index=i), SG, map)

analysis('/content/drive/Shareddrives//louvain/year/20160601-20210531/*.xlsx', '/content/drive/Shareddrives//louvain/result/20160601-20210531_reult.xlsx', folder=True)

['家飾類-毛浴枕巾', '涼被-6980以上', '食品類', '家飾類-公司自製品']
['床包-13系列', '中式床罩-8500係列', '中式床罩-1980系列', '枕頭-其他種類', '枕頭-零碼及特價品', '中式床罩-25全套組', '中式床罩-2380系列', '中式床罩-X系列', '枕頭-化纖枕', '23系列枕套', '中式床罩-6980以上', '床罩-B卡通系列', '中式床罩-2980系列', '中式床罩-3280系列', '舖棉床包組-23系列']
['家飾類-踏墊', '居家服飾-內褲', '家飾類-拖鞋']
['舖棉床包組-2980系', '薄被套-2380系列', '棉被-化纖被', '薄被套-其他', '薄被套-B系列卡通', '兩用被-零碼及特價', '薄被套-2380系列', '涼被-2980系列', '床包-零碼及特價品', '涼被-4980系列', '涼被-零碼及特價品', '薄被套-CR611/65絲', '兩用被-3280系列', '涼被-23系列', '薄被套-2980系列', '襪子', '薄被套-CR201天絲', '棉被-羽絨被', '棉被-蠶絲被', '棉被-羊毛被', '兩用被-2980系列', '涼被-5380系列', '薄被套-5980系列', '家具-其他', '床包-2980系列', '薄被套-53系列', '棉被-零碼及特價品']
['床包-X1系列', '枕頭套-X1系列', '兩用被-13系列', '床包-53系列', '枕頭套-5380系列', '床包-25系列', '中式床罩-4980系列', '枕頭套-25係列', '兩用被-53系列', '兩用被-X1系列', '床包-X2系列', '舖棉床包-25系列', '兩用被-X1系列', '2380系列床包', '中式床罩-25系列', '兩用被-2380系列', '兩用被-25系列', '保潔墊-一般']
['家飾類-抱枕', '家具-電動床', '中式床罩零碼及特', '枕頭套-零碼及特價']
['家具-特賣品', '枕頭-矽膠枕', '枕頭-乳膠枕', '保潔墊-機能性']
['兩用被-CR611/65絲', '枕頭套-P801~PIMA', '枕頭套-CR611/65絲', '兩用被-P801~PIMA', '薄被套-PIMA棉

# 畫出文氏圖

In [ ]:
#compare each clusters' people if they are same and draw the Venn diagram to the excel file
from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
from matplotlib_venn import venn3, venn3_circles
from matplotlib import pyplot as plt

def compare_two_years(cluster_1, cluster_2):
  count = 0
  for i in range(len((cluster_1['customer']))):
    for j in range(len(cluster_2['customer'])):
      if cluster_1['customer'][i] == cluster_2['customer'][j]:
        count += 1
  return count

def compare_three_years(cluster_1, cluster_2, cluster_3):
  count = 0
  for i in range(len((cluster_1['customer']))):
    for j in range(len(cluster_2['customer'])):
      if cluster_1['customer'][i] == cluster_2['customer'][j]:
        for k in range(len(cluster_3['customer'])):
          if cluster_1['customer'][i] == cluster_3['customer'][k]:
            count += 1
  return count

def draw_Venn_diagram(cluster_1, cluster_2, cluster_3, file_name, sheetname):
  plt.title('The number of customers who have purchased this group in 2018, 2019 and 2020')
  venn3(subsets = (len(cluster_1['customer']), len(cluster_2['customer']), compare_two_years(cluster_1, cluster_2), len(cluster_3['customer']), compare_two_years(cluster_1, cluster_3), compare_two_years(cluster_2, cluster_3), compare_three_years(cluster_1, cluster_2, cluster_3)), set_labels = ('The number of customers who bought this group in 2018', 'The number of customers who bought this group in 2019', 'The number of customers who bought this group in 2020'))

  plt.savefig(sheetname, bbox_inches='tight')
  plt.clf()
  wb = load_workbook(filename = file_name)
  sheet = wb.create_sheet(sheetname)

  img = openpyxl.drawing.image.Image(sheetname + '.png')
  img.anchor = 'A1'
  sheet.add_image(img)

  wb.save(file_name)

draw_Venn_diagram(result_2018[0], result_2019[5], result_2020[22], total_result_file_name, '25系列')
draw_Venn_diagram(result_2018[10], result_2019[17], result_2020[19], total_result_file_name, '雙人毯,單人毯')
draw_Venn_diagram(result_2018[7], result_2019[15], result_2020[12], total_result_file_name, '床墊,床頭,床架')
draw_Venn_diagram(result_2018[14], result_2019[12], result_2020[0], total_result_file_name, '天絲')
draw_Venn_diagram(result_2018[2], result_2019[6], result_2020[7], total_result_file_name, 'CR501,A53')
draw_Venn_diagram(result_2018[20], result_2019[22], result_2020[3], total_result_file_name, '53系列')
draw_Venn_diagram(result_2018[16], result_2019[7], result_2020[8], total_result_file_name, 'CR611,6500絲')

<Figure size 432x288 with 0 Axes>

# 畫圖(未完成)

In [ ]:
# draw the graph
pos = nx.spring_layout(G)

# color the nodes according to their partition
cmap = cm.get_cmap('viridis', max(partition.values()) + 1)
nx.draw_networkx_nodes(G, pos, partition.keys(), node_size=1, cmap=cmap, node_color=list(partition.values()))
nx.draw_networkx_edges(G, pos, alpha=0.5)
plt.show()